# Analisi dei dati del covid19 in Italia
Questa analisi dati è stata svolta prendendo i dati rilasciati dalla protezione civile sul covid19 in Italia.


- Autore: Matteo Marulli & Matteo Ghera
- Università degli studi di Firenze
- Corso di laurea magistrale in Informatica
- Curriculum: Data Science

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from path import Path
import json

In [ ]:
PROJ_DIR = Path().getcwd().parent
DATA_DIR = PROJ_DIR / "data"
os.chdir (PROJ_DIR)

with open(DATA_DIR/'dpc-covid19-ita-province.json') as f:
         d= json.load(f)
json.dumps(d)
df_province =pd.DataFrame(d)

with open(DATA_DIR/'dpc-covid19-ita-regioni.json') as f:
         d= json.load(f)
json.dumps(d)
df_regioni =pd.DataFrame(d)

In [ ]:
df_province.head()

In [ ]:
df_regioni.head()

## Descrizione dei dataset
Il dataset regioni è composto da 16 variaibili e molte variabili sono di tipo int64 e float 64.
Le variabili prese in analisi sono:
- data: è il timestamp indica quando è stato inserito il record nel database, deve essere convertito nel formato date
- stato: indica lo stato in è stato rilevato il caso (tipo qualitativo e non ordinale)
- codice_regione: codice istat per identificare una regione (tipo quantitativo e non ordinale)
- denominazione_regione: indica il nome della regione, (tipo qualitativo e non ordinale)
- lat: latidudine della regione (tipo quantitativo e ordinale)
- long: longitudine della regione (tipo quantitativo e ordinale)
- ricoverati_con_sintomi: numero di persone che sono state ricoverate nel giorno indicato da data (tipo quantitativo e ordianale)
- terapia_intesiva: numero di persone che sono state ricoverate/spostate in terapia intensiva nel giorno indicato da data (tipo quantitativo e ordianale)
- totale_ospedalizzati: numero di persone che sono state ricoverate in ospedale (somma tra quelli in tp.intensiva e ricoverati con sintomi) nel giorno indicato da data (tipo quantitativo e ordianale)
- isolamento_domiciliare: numero di persone che sono insolamento_domiciliare nel giorno indicato da data (tipo quantitativo e ordianale)
- totale_attualmente_positivi: numero di persone in isolamento e ricoverati in ospedale nel giorno indicato da data (tipo quantitativo e ordianale)
- nuovi_attualmente_positivi: differenza di persone positive al Covid19 tra il giorno attuale e il giorno passato (tipo quantitativo e ordianale)
- dimessi_guariti: persone guarite dal covid19 nella giornata indicata da data (tipo quantitativo e ordianale)
- deceduti: persone morte a causa del covid19 nella giornata indicata da data (tipo quantitativo e ordianale)
- totale_casi: totale casi positivi nella giornata indicata da data (tipo quantitativo e ordianale)
- tamponi: numerod i campioni fatti dalla regione nella giornata indicata da data (tipo quantitativo e ordianale)

Il dataset province è composto da 12 variaibili.
Le variabili prese in analisi sono:
- data: è il timestamp indica quando è stato inserito il record nel database, deve essere convertito nel formato date (tipo qualitativo e non ordinale)
- stato: indica lo stato, verrà cancellata nel preprocessing (tipo qualitativo e non ordinale)
- codice_regione: codice istat per identificare una regione (tipo quantitativo e non ordinale)
- denominazione_regione: indica il nome della regione (tipo qualitativo e non ordinale)
- lat: latidudine della città (tipo quantitativo e ordinale)
- long: longitudine della città (tipo quantitativo e ordinale)
- codice_provincia: codice istat per identificare una provincia (tipo quantitativo e non ordinale)
- denominanzione_provincia: indica il nome della provincia (tipo qualitativo e non ordinale)
- sigla_provincia: sigla della provincia usate per le targe auto (tipo qualitativo e non ordinale)
- totale_casi: persone infette da covid19 (tipo quantitativo e ordianale)
- note_it: note informative in italiano (tipo qualitativo e non ordinale)
- note_en: note informative in inglese (tipo qualitativo e non ordinale)
- casi_testati: numero di persone sottoposte al test per covid-19


Le colonne in comune nei due dataset sono:
- data
- stato
- note_en
- note_it
- codice_regione
- totale_casi
- denominazione_regione
- lat
- long

Le colonne candidate per eseguire la join sono data e denominazione_regione

In [ ]:
set(df_regioni.columns).intersection(set(df_province.columns))

In [ ]:
joinDF = pd.merge(df_province, df_regioni, on= ['data', 'codice_regione'])
joinDF.drop(axis=1, columns = joinDF.filter(regex='_y').columns, inplace = True)

In [ ]:
joinDF.head().T

In [ ]:
joinDF.info(memory_usage=True)

In [ ]:
joinDF.describe()

In [ ]:
plt.figure(figsize=(10,5), dpi = 150)
sns.heatmap(joinDF.corr(), annot=True, linewidths= 0.5 , cmap = 'coolwarm')

In [ ]:
#sns.pairplot(data=joinDF)

## Data cleaning

In [ ]:
joinDF.columns = joinDF.columns.str.replace('_x','')

In [ ]:
joinDF['data'] = pd.to_datetime(joinDF['data'], format='%Y-%m-%d') 

In [ ]:
joinDF.drop(columns=['stato', 'codice_regione', 'note_en', 'note_it', 'codice_regione', 'codice_provincia','sigla_provincia'], axis=1, inplace = True)

In [ ]:
joinDF['casi_testati'].fillna(joinDF['casi_testati'].mean(), inplace = True)

In [ ]:
multi = pd.MultiIndex.from_tuples([(regione, data) for regione,data in zip(joinDF['denominazione_regione'],joinDF['data'])], names =['denominazione_regione', 'data'])

In [ ]:
joinDF.index = multi

In [ ]:
joinDF

In [ ]:
joinDF.pivot_table(index = ['denominazione_regione', 'denominazione_provincia'])

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
titanic['survived'](sex).mean()

In [ ]:
titanic[titanic['sex']=='male']['survived'].mean()

In [ ]:
titanic.groupby('sex')[['survived']].mean()

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack() # calcolo la media dei sopravissuti per ogni classe divisa tra maschi e femminie

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class') #calcola la media dei sopravissuti per ogni classe divisi per maschi e femmini di età 0, 18 e 18,80

In [ ]:
joinDF.pivot_table('totale_positivi', index = ['denominazione_regione', 'denominazione_provincia'], columns=['data'], aggfunc='sum')

In [ ]:
plt.figure(figsize=(12,6), dpi = 200)
joinDF.pivot_table('totale_positivi', index=['denominazione_regione', 'data'], aggfunc='sum' )

In [ ]:
casiPositviPerRegioni = joinDF.pivot_table('totale_positivi', index=['data'], columns=['denominazione_regione'], aggfunc='sum' )
casiPositviPerRegioni.plot(figsize=(14,10), marker='s', fontsize=22)
plt.title('Grafico dei casi positivi per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di casi positivi', size = 22 )
plt.legend(loc=(1,0.35))

In [ ]:
tamponiPerRegioni = joinDF.pivot_table('tamponi', index=['data'], columns=['denominazione_regione'], aggfunc='sum' )
tamponiPerRegioni.plot(figsize=(14,10), marker='s', fontsize=22)
plt.title('Grafico dei tamponi fatti per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di tamponi fatti', size = 22 )
plt.legend(loc=(1,0.35))

In [ ]:
decedutiPerRegioni = joinDF.pivot_table('deceduti', index=['data'], columns=['denominazione_regione'], aggfunc='count' )
decedutiPerRegioni.plot(figsize=(14,10), marker='s', fontsize=22)
plt.title('Grafico dei deceduti per regioni', size = 22)
plt.xlabel('Data',size = 22)
plt.ylabel('Numero di deceduti', size = 22 )
plt.legend(loc=(1,0.35))

In [ ]:
decedutiPerRegioni['Lombardia']

In [ ]:
tamponiPerRegioni

In [ ]:
joinDF[joinDF['denominazione_regione']=='Abruzzo'].head(20).T